In [9]:
# --- Markdown ---
# ## 1. Setup and Data Loading
#
# Import necessary libraries and load data from CSV files.
# Prepare data structures for both the distance calculation function and the Pyomo model.
# --- End Markdown ---

# --- Python Code ---
%pip install pyomo
import pyomo.environ as pyo
import pandas as pd
import requests   # To make HTTP requests to OSRM API
import json       # To parse JSON responses
import time       # To potentially add delays for rate limiting
import sys        # For error messages
import math       # For fallback distance calc if needed

# --- Configuration ---
# OSRM Server URL (Using the public demo server)
OSRM_BASE_URL = "http://router.project-osrm.org"

# Define the cost per kilometer (as per notebook)
C_KM = 20700 # COP/km

# --- Load Data from CSV ---
try:
    depots_df = pd.read_csv("case_1_base/Depots.csv")
    clients_df = pd.read_csv("case_1_base/Clients.csv")
    vehicles_df = pd.read_csv("case_1_base/Vehicles.csv")
    print("CSV files loaded successfully.")
except FileNotFoundError as e:
    print(f"ERROR: Could not find CSV file: {e}. Make sure the files are in the same directory.", file=sys.stderr)
    sys.exit(1) # Exit if data files are missing
except Exception as e:
    print(f"ERROR: An error occurred while loading CSV files: {e}", file=sys.stderr)
    sys.exit(1)

# --- Process Data for Pyomo & the compute_distance_matrix function ---

# A. Pyomo-specific Sets
set_I_data = [f'D{depot_id}' for depot_id in depots_df['DepotID']]
set_J_data = [f'C{client_id}' for client_id in clients_df['ClientID']]
set_K_data = [f'V{i+1}' for i in range(len(vehicles_df))] # Vehicle IDs V1, V2, ...
set_N_data = set_I_data + set_J_data

# B. Pyomo-specific Parameters
param_A_data = {f'D{depots_df.loc[i, "DepotID"]}': float('inf') for i in depots_df.index}
print("WARNING: Depot capacities (A_i) not found in Depots.csv. Assuming infinite capacity.")

param_D_data = {f'C{clients_df.loc[i, "ClientID"]}': clients_df.loc[i, 'Product']
                for i in clients_df.index}
param_Q_data = {f'V{i+1}': vehicles_df.loc[i, 'Capacity'] for i in vehicles_df.index}
param_R_data = {f'V{i+1}': vehicles_df.loc[i, 'Range'] for i in vehicles_df.index}
param_n_cust_data = len(set_J_data)

# C. Data Dictionary for compute_distance_matrix function
data = {}
data['N'] = set_N_data # Set of Node IDs
data['UbicacionNodo'] = {} # Node coordinates {NodeID: (lat, lon)}
data['CV'] = list(vehicles_df['VehicleType'].unique()) # List of unique vehicle types
if 'Aereo' in data['CV']:
    print("Warning: 'Aereo' vehicle type found, will be excluded from OSRM distance calculation as per function logic.")

# Populate coordinates dictionary (lat, lon)
for i in depots_df.index:
    node_id = f'D{depots_df.loc[i, "DepotID"]}'
    data['UbicacionNodo'][node_id] = (depots_df.loc[i, 'Latitude'], depots_df.loc[i, 'Longitude'])
for i in clients_df.index:
    node_id = f'C{clients_df.loc[i, "ClientID"]}'
    data['UbicacionNodo'][node_id] = (clients_df.loc[i, 'Latitude'], clients_df.loc[i, 'Longitude'])

# Initialize the nested distance dictionary expected by the function
data['d'] = {tv: {u: {} for u in data['N']} for tv in data['CV']}

# D. Placeholder Parameters for Pyomo (will be filled by OSRM results)
param_d_data = {} # Will store {(u, v): distance_km}
param_c_data = {} # Will store {(u, v): cost_cop}

print("--- Data Prepared for Pyomo and Distance Function ---")
print(f"Set I (Depots): {set_I_data}")
print(f"Set J (Customers): {set_J_data}")
print(f"Set K (Vehicles): {set_K_data}")
print(f"Set N (Nodes): {data['N']}")
print(f"Param D (Demands): {param_D_data}")
print(f"Param Q (Veh. Caps): {param_Q_data}")
print(f"Param R (Veh. Ranges): {param_R_data}")
print(f"Param n_cust: {param_n_cust_data}")
print(f"Vehicle Types (CV): {data['CV']}")
# print(f"Node Locations (UbicacionNodo): {data['UbicacionNodo']}") # Can be long
print("--- End Data Preparation ---")
# --- End Python Code ---

Note: you may need to restart the kernel to use updated packages.
CSV files loaded successfully.
--- Data Prepared for Pyomo and Distance Function ---
Set I (Depots): ['D1']
Set J (Customers): ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']
Set K (Vehicles): ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24']
Set N (Nodes): ['D1', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']
Param D (Demands): {'C1': np.int64(13), 'C2': np.int64(15), 'C3': np.int64(12), 'C4': np.int64(15), 'C5': np.int64(20), 'C6': np.int64(17), 'C7': np.int64(17), 'C8': np.int64(20), 'C9': np.int64(20), 'C10': np.int64(15), 'C11': np.int64(17), 'C12': np.int64(12), 'C13': np.int64(21), 'C14

In [ ]:
# # --- Markdown ---
# # ## 2. Distance and Cost Matrix Calculation using OSRM
# #
# # Define and execute the function provided by the user to compute distances using the OSRM API.
# # Extract the results into the format required by the Pyomo model.
# # --- End Markdown ---

# # --- Python Code ---
# def compute_distance_matrix(data):
#     """
#     Computes the distance matrix using OSRM table service for ground vehicles.
#     Modifies the data['d'] dictionary in place.
#     Input 'data' dictionary requires:
#         - data['N']: List of node IDs
#         - data['UbicacionNodo']: Dict mapping NodeID -> (lat, lon)
#         - data['CV']: List of vehicle types (e.g., ['Gas Car', 'Aereo'])
#         - data['d']: Pre-initialized nested dict {tv: {u: {v: 0}}}
#     """
#     nodes = data['N']
#     node_coords_dict = data['UbicacionNodo']
#     # Extract coords in the order of nodes list for consistent indexing
#     node_coords_list = [node_coords_dict[i] for i in nodes]

#     # Prepare coordinates for the OSRM API in lon,lat format
#     coords_str = ';'.join([f"{lon},{lat}" for lat, lon in node_coords_list])

#     # OSRM distance computation for all ground vehicles (single call)
#     # Exclude 'Aereo' type as per the function's logic
#     ground_vehicle_types = [cv for cv in data['CV'] if cv != 'Aereo']
#     if not ground_vehicle_types:
#          print("Warning: No ground vehicle types found in data['CV'] to calculate distances for.", file=sys.stderr)
#          return # Nothing to calculate

#     url = f"{OSRM_BASE_URL}/table/v1/driving/{coords_str}"
#     print(f"Requesting distances from OSRM: {url[:100]}...")
#     params = {
#         'annotations': 'distance'
#     }
#     try:
#         # Single API call
#         response = requests.get(url, params=params, timeout=90) # Increased timeout
#         response.raise_for_status()  # Raise an HTTPError if status != 200
#         result = response.json()

#         if result['code'] != 'Ok':
#             raise ValueError(f"OSRM API Error: {result['code']} - {result.get('message', 'No message')}")
#         if 'distances' not in result:
#             raise ValueError("No 'distances' key found in OSRM response")

#         distances = result['distances'] # This is a list of lists

#         # Check matrix dimensions
#         if len(distances) != len(nodes) or (len(distances) > 0 and len(distances[0]) != len(nodes)):
#              raise ValueError(f"OSRM returned distance matrix of unexpected size: {len(distances)}x{len(distances[0]) if len(distances)>0 else 0}, expected {len(nodes)}x{len(nodes)}")


#         # Populate the data['d'] dictionary
#         for tv in ground_vehicle_types:
#             for i, from_node in enumerate(nodes):
#                 for j, to_node in enumerate(nodes):
#                     if distances[i] is None or distances[i][j] is None:
#                          print(f"Warning: OSRM returned null distance for ({from_node}, {to_node}). Setting to infinity.", file=sys.stderr)
#                          data['d'][tv][from_node][to_node] = float('inf')
#                     else:
#                         distance = distances[i][j] / 1000.0  # Convert meters to kilometers
#                         data['d'][tv][from_node][to_node] = distance
#         print("OSRM distance calculation successful.")

#     except requests.exceptions.Timeout:
#         print(f"Error: OSRM request timed out.", file=sys.stderr)
#         print("Setting distances to infinity.", file=sys.stderr)
#         for tv in ground_vehicle_types:
#             for i in nodes:
#                 for j in nodes:
#                     data['d'][tv][i][j] = float('inf')
#     except Exception as e:
#         print(f"Error fetching or processing distances for ground vehicles: {e}", file=sys.stderr)
#         print("Setting distances to infinity.", file=sys.stderr)
#         # Default to a large number (infinity) in case of any failure
#         for tv in ground_vehicle_types:
#             for i in nodes:
#                 for j in nodes:
#                     data['d'][tv][i][j] = float('inf')

# # --- Execute Distance Calculation ---
# print("Calculating distances using the provided OSRM function...")
# compute_distance_matrix(data)

# # --- Extract distances into Pyomo parameter format ---
# # Assuming distance is the same for all ground vehicle types in the model
# # We take the distances calculated for the first ground vehicle type
# ground_vehicle_types = [cv for cv in data['CV'] if cv != 'Aereo']
# if ground_vehicle_types:
#     first_ground_type = ground_vehicle_types[0]
#     if first_ground_type in data['d']:
#         for u in data['N']:
#             for v in data['N']:
#                 # Check if the inner dictionary exists before accessing
#                 if u in data['d'][first_ground_type] and v in data['d'][first_ground_type][u]:
#                     param_d_data[u, v] = data['d'][first_ground_type][u][v]
#                     param_c_data[u, v] = C_KM * param_d_data[u, v]
#                 else:
#                     # Handle cases where a node pair might be missing (shouldn't happen with table)
#                     print(f"Warning: Missing distance data for ({u}, {v}). Setting to infinity.", file=sys.stderr)
#                     param_d_data[u, v] = float('inf')
#                     param_c_data[u, v] = float('inf')
#         print(f"Extracted distances for Pyomo model using type '{first_ground_type}'.")
#     else:
#          print(f"Error: No distance data found for vehicle type '{first_ground_type}' after OSRM call. Using zeros.", file=sys.stderr)
#          for u in data['N']:
#             for v in data['N']:
#                 param_d_data[u, v] = 0.0
#                 param_c_data[u, v] = 0.0
# else:
#     print("Error: No ground vehicle types defined. Cannot extract distances. Using zeros.", file=sys.stderr)
#     for u in data['N']:
#         for v in data['N']:
#             param_d_data[u, v] = 0.0
#             param_c_data[u, v] = 0.0

# # Display a small part of the extracted distance matrix for verification
# print("\n--- Sample Extracted Distances (km) for Pyomo ---")
# sample_nodes = set_N_data[:4] # Show first 4 nodes
# for u in sample_nodes:
#     for v in sample_nodes:
#         dist_val = param_d_data.get((u, v), float('inf'))
#         # Check for infinity before formatting
#         dist_str = f"{dist_val:.2f}" if dist_val != float('inf') else "inf"
#         print(f"d({u}, {v}) = {dist_str}", end = ' | ')
#     print()
# print("--- End Distance Sample ---")

# # Reminder about public server limitations
# print("\nNOTE: Using the public OSRM demo server. This has rate limits and is not suitable for large-scale or production use.")
# print("Consider setting up a local OSRM instance for better performance and reliability.")
# # --- End Python Code ---

# --- Python Code ---
import math, json, os, requests

OSRM_URL   = "https://router.project-osrm.org/table/v1/driving/"
CACHE_FILE = "osrm_km_cache.json"     # se reutiliza entre ejecuciones
C_KM       = 20700                    # COP por km

# ------------------------------------------------------------------
# 1)  Construir lista de nodos y diccionario de coordenadas
#     (ya los tienes como set_N_data y data['UbicacionNodo'])
# ------------------------------------------------------------------
N      = data['N']
coords = data['UbicacionNodo']        # {node: (lat, lon)}

# ------------------------------------------------------------------
# 2)  Cargar caché (claves "u|v")
# ------------------------------------------------------------------
dist_cache = {}
if os.path.exists(CACHE_FILE):
    try:
        with open(CACHE_FILE) as f:
            raw = json.load(f)
        dist_cache = {tuple(k.split("|")): v for k, v in raw.items()}
        print(f"  Distancias cargadas de caché ({len(dist_cache)} pares).")
    except json.JSONDecodeError:
        print("  Caché dañada, se ignorará.")
        dist_cache = {}

# ------------------------------------------------------------------
# 3)  Identificar pares faltantes y, si hay, llamar a OSRM
# ------------------------------------------------------------------
missing = [(u, v) for u in N for v in N if (u, v) not in dist_cache]

if missing:
    print(f"Consultando OSRM para {len(missing)} pares…")
    MAX_COORDS = 100                                   # límite del servidor demo
    for start in range(0, len(N), MAX_COORDS):
        sub_nodes  = N[start:start+MAX_COORDS]
        coord_str  = ';'.join(f"{coords[n][1]},{coords[n][0]}" for n in sub_nodes)
        try:
            r = requests.get(OSRM_URL + coord_str,
                             params={"annotations": "distance"},
                             timeout=60)
            r.raise_for_status()
            matrix = r.json()["distances"]             # metros
        except Exception as e:
            print("  Error OSRM:", e, "→ se usa ∞")
            matrix = [[math.inf]*len(sub_nodes) for _ in sub_nodes]

        for i, u in enumerate(sub_nodes):
            for j, v in enumerate(sub_nodes):
                dist_cache[(u, v)] = matrix[i][j] / 1000.0   # a km

    # guardar caché
    with open(CACHE_FILE, "w") as f:
        json.dump({f"{u}|{v}": km for (u, v), km in dist_cache.items()}, f)
    print(" Caché OSRM actualizada.")
else:
    print("No faltaban distancias nuevas.")

# ------------------------------------------------------------------
# 4)  Rellenar param_d_data y param_c_data para Pyomo
# ------------------------------------------------------------------
param_d_data.clear()
param_c_data.clear()
for u in N:
    for v in N:
        km = dist_cache.get((u, v), math.inf)
        param_d_data[u, v] = km
        param_c_data[u, v] = km * C_KM
print(f"➜  Matriz de distancias y costos lista ({len(N)} × {len(N)} pares).")

# (opcional) mostrar un subconjunto
print("\nMuestra (primeros 4 nodos):")
for u in N[:4]:
    for v in N[:4]:
        print(f"d({u},{v})={param_d_data[u,v]:.2f} km, "
              f"c={param_c_data[u,v]:,.0f}", end="  |  ")
    print()
# --- End Python Code ---


ℹ️  Distancias cargadas de caché (966 pares).
No faltaban distancias nuevas.
➜  Matriz de distancias y costos lista (25 × 25 pares).

Muestra (primeros 4 nodos):
d(D1,D1)=0.00 km, c=0  |  d(D1,C1)=23.34 km, c=483,177  |  d(D1,C2)=8.91 km, c=184,369  |  d(D1,C3)=7.97 km, c=165,008  |  
d(C1,D1)=22.72 km, c=470,306  |  d(C1,C1)=0.00 km, c=0  |  d(C1,C2)=14.26 km, c=295,105  |  d(C1,C3)=19.40 km, c=401,586  |  
d(C2,D1)=10.13 km, c=209,741  |  d(C2,C1)=12.67 km, c=262,315  |  d(C2,C2)=0.00 km, c=0  |  d(C2,C3)=6.81 km, c=141,021  |  
d(C3,D1)=11.32 km, c=234,353  |  d(C3,C1)=15.95 km, c=330,169  |  d(C3,C2)=6.48 km, c=134,198  |  d(C3,C3)=0.00 km, c=0  |  


In [11]:
# --- Markdown ---
# ## 3. Pyomo Model Definition
#
# Define the Pyomo model structure (sets, parameters, variables, objective, constraints)
# using the data loaded from CSV and distances calculated via OSRM.
# --- End Markdown ---

# --- Python Code ---
# Create a concrete model
model = pyo.ConcreteModel(name="LogistiCo_VRP_CSV_OSRM_UserFunc")

# --- Sets ---
model.I = pyo.Set(initialize=set_I_data, doc="Distribution Centers")
model.J = pyo.Set(initialize=set_J_data, doc="Customers")
model.K = pyo.Set(initialize=set_K_data, doc="Vehicles")
model.N = pyo.Set(initialize=set_N_data, doc="All Nodes")

# --- Parameters ---
model.A = pyo.Param(model.I, initialize=param_A_data, default=float('inf'), doc="Capacity of CD i")
model.D = pyo.Param(model.J, initialize=param_D_data, doc="Demand of customer j")
model.Q = pyo.Param(model.K, initialize=param_Q_data, doc="Capacity of vehicle k")
model.R = pyo.Param(model.K, initialize=param_R_data, doc="Range of vehicle k")
# Initialize with the extracted OSRM distances, default to infinity if missing
model.d = pyo.Param(model.N, model.N, initialize=param_d_data, default=float('inf'), doc="Distance from u to v (km)")
model.c = pyo.Param(model.N, model.N, initialize=param_c_data, default=float('inf'), doc="Cost from u to v (COP)")
model.n_cust = pyo.Param(initialize=param_n_cust_data, doc="Number of customers")

# --- Decision Variables ---
model.x = pyo.Var(model.N, model.N, model.K, within=pyo.Binary, doc="Vehicle k travels from u to v")
model.y = pyo.Var(model.I, model.K, within=pyo.Binary, doc="Vehicle k assigned to CD i")
model.u = pyo.Var(model.J, model.K, within=pyo.NonNegativeReals, bounds=(0, model.n_cust), doc="MTZ auxiliary variable") # Added bounds for clarity

# --- Objective Function ---
def objective_rule(mod):
    # Ensure cost is not infinite when calculating objective
    cost = sum(mod.c[u, v] * mod.x[u, v, k]
               for k in mod.K for u in mod.N for v in mod.N
               if u != v and mod.c[u, v] != float('inf'))
    # Add penalty for using arcs with infinite cost (should not happen if feasible)
    penalty = sum(1e9 * mod.x[u, v, k]
                   for k in mod.K for u in mod.N for v in mod.N
                   if u != v and mod.c[u, v] == float('inf'))
    return cost + penalty
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize, doc="Minimize total cost")

# --- Constraints ---
# Constraint 1: Customer Coverage
def customer_coverage_rule(mod, j):
    return sum(mod.x[u, j, k] for k in mod.K for u in mod.N if u != j) == 1
model.customer_coverage = pyo.Constraint(model.J, rule=customer_coverage_rule, doc="Each customer visited exactly once")

# Constraint 2a: Flow Conservation at Customer Nodes
def flow_conservation_customer_rule(mod, j, k):
    in_flow = sum(mod.x[u, j, k] for u in mod.N if u != j)
    out_flow = sum(mod.x[j, v, k] for v in mod.N if v != j)
    return in_flow == out_flow
model.flow_conservation_customer = pyo.Constraint(model.J, model.K, rule=flow_conservation_customer_rule, doc="Flow conservation at customer nodes")

# Constraint 2b: Vehicle Departure from Assigned Depot
def vehicle_departure_rule(mod, i, k):
    return sum(mod.x[i, v, k] for v in mod.N if v != i) == mod.y[i, k]
model.vehicle_departure = pyo.Constraint(model.I, model.K, rule=vehicle_departure_rule, doc="Vehicle leaves assigned depot")

# Constraint 2c: Vehicle Return to Assigned Depot
def vehicle_return_rule(mod, i, k):
    return sum(mod.x[u, i, k] for u in mod.N if u != i) == mod.y[i, k]
model.vehicle_return = pyo.Constraint(model.I, model.K, rule=vehicle_return_rule, doc="Vehicle returns to assigned depot")

# Constraint 2d: Vehicle Assignment Constraint
def vehicle_assignment_rule(mod, k):
    return sum(mod.y[i, k] for i in mod.I) <= 1
model.vehicle_assignment = pyo.Constraint(model.K, rule=vehicle_assignment_rule, doc="Each vehicle assigned to at most one depot")


# Constraint 3: Vehicle Capacity
def vehicle_capacity_rule(mod, k):
    demand_served_by_k = sum(mod.D[j] * sum(mod.x[u, j, k] for u in mod.N if u != j) for j in mod.J)
    return demand_served_by_k <= mod.Q[k]
model.vehicle_capacity = pyo.Constraint(model.K, rule=vehicle_capacity_rule, doc="Vehicle capacity constraint")

# Constraint 4: Vehicle Range (Autonomy)
def vehicle_range_rule(mod, k):
    # Ensure distance is not infinite when calculating range
    dist_traveled = sum(mod.d[u, v] * mod.x[u, v, k]
                       for u in mod.N for v in mod.N
                       if u != v and mod.d[u,v] != float('inf'))
    # Add a check to ensure no infeasible arcs are used
    infeasible_arc_used = sum(mod.x[u, v, k]
                              for u in mod.N for v in mod.N
                              if u != v and mod.d[u,v] == float('inf'))
    if pyo.value(infeasible_arc_used) > 0.1: # If solver uses an infinite arc
         return pyo.Constraint.Infeasible
    return dist_traveled <= mod.R[k]
model.vehicle_range = pyo.Constraint(model.K, rule=vehicle_range_rule, doc="Vehicle range constraint")

# Constraint 5: Distribution Center Capacity (Omitted)
print("Skipping CD Capacity Constraint (5).")

# Constraint 6: Subtour Elimination (MTZ)
def subtour_elimination_rule(mod, j, j_prime, k):
    if j == j_prime:
        return pyo.Constraint.Skip
    # Make sure arc exists before applying constraint
    if mod.d[j,j_prime] == float('inf'): # If no route exists between j and j', x should be 0
        return mod.x[j,j_prime,k] == 0
    return mod.u[j, k] - mod.u[j_prime, k] + mod.n_cust * mod.x[j, j_prime, k] <= mod.n_cust - 1
model.subtour_elimination = pyo.Constraint(model.J, model.J, model.K, rule=subtour_elimination_rule, doc="MTZ subtour elimination")

# Constraint 7: Prevent trivial loops (x_uuk = 0)
def no_trivial_loops_rule(mod, u, k):
    return mod.x[u, u, k] == 0
model.no_trivial_loops = pyo.Constraint(model.N, model.K, rule=no_trivial_loops_rule, doc="Prevent travel from a node to itself")

print("Pyomo model structure defined using data from CSVs and OSRM API (User Func).")
# --- End Python Code ---

Skipping CD Capacity Constraint (5).
Pyomo model structure defined using data from CSVs and OSRM API (User Func).


In [ ]:
# --- Markdown ---
# ## 4. Solve the Model and Display Results
#
# Use a selected MILP solver (e.g., GLPK, CBC) to find the optimal solution
# for the VRP model and display the results, including cost, assignments, and routes.
# --- End Markdown ---

# --- Python Code ---
solver_name = 'gurobi' # Or 'cbc'
try:
    solver = pyo.SolverFactory(solver_name)
    if not solver.available():
        raise RuntimeError(f"Solver '{solver_name}' not found or not executable.")
except Exception as e:
    print(f"ERROR: Could not find or initialize solver '{solver_name}'. Please install it and ensure it's in your PATH. Error: {e}", file=sys.stderr)
    print("You can try installing GLPK or CBC (e.g., using conda).", file=sys.stderr)
    sys.exit(1)


print(f"\nSolving the model using {solver_name}...")
# Add a time limit, e.g., 5 minutes, especially for larger problems
# results = solver.solve(model, tee=True, timelimit=300)
results = solver.solve(model, tee=True) # tee=True shows solver output

# --- Display Results ---

print("\n--- Solver Results ---")
print(results) # Print detailed solver results object

if results.solver.termination_condition == pyo.TerminationCondition.optimal or \
   (results.solver.termination_condition == pyo.TerminationCondition.feasible and len(results.solution) > 0) or \
   (results.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit and len(results.solution) > 0):

    if results.solver.termination_condition == pyo.TerminationCondition.optimal:
        print("\n--- Optimal Solution Found ---")
    elif results.solver.termination_condition == pyo.TerminationCondition.maxTimeLimit:
         print("\n--- Time Limit Reached - Feasible Solution Found ---")
    else:
        print("\n--- Feasible Solution Found (may not be optimal) ---")

    print(f"Minimum Total Cost: {pyo.value(model.objective):,.2f} COP")

    print("\n--- Vehicle Assignments (y_ik) ---")
    assigned_vehicles_count = 0
    assignments = {}
    for k in model.K:
        assignments[k] = 'Unassigned'
        for i in model.I:
            # Use a tolerance for checking binary variable values
            if pyo.value(model.y[i, k], exception=False) is not None and pyo.value(model.y[i, k]) > 0.5:
                print(f"Vehicle {k} starts from CD {i}")
                assignments[k] = i
                assigned_vehicles_count += 1
                break
        if assignments[k] == 'Unassigned':
             print(f"Vehicle {k} is not used.")
    if assigned_vehicles_count == 0:
        print("No vehicles were assigned to any routes.")

    print("\n--- Routes (x_uvk) ---")
    total_distance = 0
    total_demand_served = 0
    vehicles_used = set()

    for k in model.K:
        start_depot = assignments.get(k)
        if start_depot != 'Unassigned':
            print(f"\nRoute for Vehicle {k} (from {start_depot}):")
            route = [start_depot]
            current_node = start_depot
            route_distance = 0
            route_demand = 0
            visited_nodes = {start_depot}
            route_found = False
            max_steps = len(model.N) + 2 # Increased safety margin

            for step in range(max_steps):
                next_node_found = False
                possible_next = []
                for v in model.N:
                    if current_node != v:
                         # Check value safely, handling potential None if variable wasn't solved
                         x_val = pyo.value(model.x[current_node, v, k], exception=False)
                         if x_val is not None and x_val > 0.5:
                            possible_next.append(v)
                            next_node_found = True

                if len(possible_next) > 1 and current_node != start_depot:
                     print(f"  ERROR: Multiple next steps found from {current_node} for vehicle {k}: {possible_next}")
                     route = [start_depot, "Error"] # Mark route as problematic
                     break
                elif not next_node_found and current_node != start_depot:
                     print(f"  ERROR: No next step found from {current_node} for vehicle {k}. Incomplete route.")
                     route.append("Error")
                     break
                elif not next_node_found and current_node == start_depot:
                    print(f"  Vehicle {k} assigned but seems to have no outgoing route from {start_depot}.")
                    route = [start_depot]
                    break

                if next_node_found:
                    next_node = possible_next[0]
                    arc_dist = model.d[current_node, next_node]
                    if arc_dist == float('inf'):
                        print(f"  ERROR: Route uses an infeasible arc from {current_node} to {next_node}")
                        route.append(f"{next_node}(inf)")
                        route_found = False # Mark as incomplete due to error
                        break
                    route_distance += arc_dist
                    route.append(next_node)
                    visited_nodes.add(next_node)

                    if next_node in model.J:
                        route_demand += model.D[next_node]

                    current_node = next_node

                    if current_node == start_depot:
                        route_found = True
                        break

                if step == max_steps - 1 and not route_found:
                    print(f"  ERROR: Route for vehicle {k} did not return to depot within {max_steps} steps.")
                    route.append("...") # Indicate incomplete


            # Print reconstructed route and details
            if route_found:
                 print(f"  {' -> '.join(route)}")
                 print(f"  Distance: {route_distance:.2f} km (Max: {model.R[k]:.2f})")
                 print(f"  Demand: {route_demand:.2f} kg (Capacity: {model.Q[k]:.2f})")
                 total_distance += route_distance
                 total_demand_served += route_demand
                 vehicles_used.add(k)
                 # Validation checks
                 if route_distance > model.R[k] + 1e-6: print(f"  WARNING: Vehicle {k} exceeded range!")
                 if route_demand > model.Q[k] + 1e-6: print(f"  WARNING: Vehicle {k} exceeded capacity!")
            elif len(route) > 1 : # Print incomplete/error routes if they exist
                 print(f"  Route Issue: {' -> '.join(route)}")
            # Else: already handled cases where vehicle didn't leave depot


    print("\n--- Overall Summary ---")
    print(f"Total distance covered by all vehicles: {total_distance:.2f} km")
    print(f"Total demand served: {total_demand_served:.2f} kg")
    print(f"Total expected demand: {sum(param_D_data.values()):.2f} kg")
    print(f"Number of vehicles used: {len(vehicles_used)} out of {len(model.K)}")
    if abs(total_demand_served - sum(param_D_data.values())) > 1e-6:
        print("WARNING: Total demand served does not match total expected demand!")


elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
     print("\n--- Model is Infeasible ---")
     print("The solver determined that there is no solution that satisfies all constraints.")
     print("Check constraints (capacities, ranges) and data for potential issues.")
     # Consider using Pyomo's tools for analyzing infeasibility if needed:
     # from pyomo.util.infeasible import log_infeasible_constraints
     # log_infeasible_constraints(model)

else:
    print("\n--- Solver did not find an Optimal or Feasible Solution ---")
    print(f"Solver Status: {results.solver.status}")
    print(f"Termination Condition: {results.solver.termination_condition}")
    print("Check solver logs and model formulation for errors.")
# --- End Python Code ---


Solving the model using gurobi...
Set parameter Username
Set parameter LicenseID to value 2654934
Academic license - for non-commercial use only - expires 2026-04-21
Read LP format model from file C:\Users\VivoBook\AppData\Local\Temp\tmp1hzy9xww.pyomo.lp
Reading time = 0.20 seconds
x1: 14568 rows, 15600 columns, 111264 nonzeros
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14568 rows, 15600 columns and 111264 nonzeros
Model fingerprint: 0x2f26f15a
Variable types: 576 continuous, 15024 integer (15024 binary)
Coefficient statistics:
  Matrix range     [7e-01, 3e+01]
  Objective range  [1e+04, 6e+05]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+02]
Presolve removed 624 rows and 600 columns
Presolve time: 0.43s
Presolved: 13944 rows, 15000 columns, 1102